# Подготовка данных для предсказания и замены чисел вместо пропусков

## Импорт библиотек

In [1]:
!pip install miceforest

   ---------------------------------------- 0.0/1.4 MB ? eta -:--:--
   ------------------------------------ --- 1.3/1.4 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------- 1.4/1.4 MB 8.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/25.1 MB ? eta -:--:--
   ---- ----------------------------------- 2.9/25.1 MB 12.9 MB/s eta 0:00:02
   -------- ------------------------------- 5.2/25.1 MB 12.7 MB/s eta 0:00:02
   ---------- ----------------------------- 6.3/25.1 MB 9.9 MB/s eta 0:00:02
   ----------------- ---------------------- 11.0/25.1 MB 13.5 MB/s eta 0:00:02
   ---------------------- ----------------- 13.9/25.1 MB 13.6 MB/s eta 0:00:01
   -------------------------- ------------- 16.5/25.1 MB 13.5 MB/s eta 0:00:01
   ------------------------------ --------- 19.4/25.1 MB 13.5 MB/s eta 0:00:01
   ----------------------------------- ---- 22.3/25.1 MB 13.4 MB/s eta 0:00:01
   ---------------------------------------  24.9/25.1 MB 13.5 MB/s eta 0:00:01
 

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import miceforest as mf
from sklearn.base import clone
from itertools import combinations
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error, r2_score

##Функции/Классы

### **функции**

In [3]:
def nan_to_median(series: pd.Series):
    """
    получает pd.Series с пропусками.
    возвращает pd.Series с медианой вместо пропусков.
    """
    median_val = series.median()
    return series.fillna(median_val)


####################################################################################################


def radical_iqr_filter(df: pd.DataFrame, column: str, lower_bound=True, upper_bound=True, fill_nan=False, drop_nan_values=False, multp=3):
    """
    гибкая функция замены выбросов на NaN с помощью настраиваемого интерквартильного размаха

    Аргументы
        df: DataFrame содержащий данные, которые нужно отфильтровать.

        column: Название столбца в DataFrame df, в которомом будет производиться фильтрация.
        Функция будет рассчитывать IQR именно для этого столбца.

        lower_bound:  Булевый флаг, определяющий, следует ли отфильтровывать значения,
        лежащие ниже нижней границы, рассчитанной на основе IQR. То есть, если
        lower_bound = False, то все выбросы(если они есть) будут игнорироваться.

        upper_bound: Булевый флаг, определяющий, следует ли отфильтровывать значения,
        лежащие выше верхней границы, рассчитанной на основе IQR.

        multp: Множитель, используемый для расчета границ фильтрации.
    """
    df_copy = df.copy()

    # Заменяет пропущенные значения меданной в столбце column, если fill_nan == True
    if fill_nan:
        df_copy[column] = nan_to_median(df_copy[column])
    if drop_nan_values:
        df_copy[column].dropna()

    #получение 25 и 75 процентилей
    q1 = df_copy[column].quantile(0.25)
    q3 = df_copy[column].quantile(0.75)

    # Вычисление рамок для фильтрации
    iqr = (q3 - q1) * multp
    low_bound = q1 - iqr
    up_bound = q3 + iqr
    outlines = 0 # индексы, значения строк столбца которых необходимо заменить медианой


    # Считаем за выбросы, которые заходят за up_bound или/и low_bound.
    if lower_bound and upper_bound:
        outlines = df_copy[(df_copy[column] < low_bound) | (df_copy[column] > up_bound)].index
    elif lower_bound:
        outlines = df_copy[df_copy[column] < low_bound].index
    elif upper_bound:
        outlines = df_copy[df_copy[column] > up_bound].index

    # Замена выбросов на np.nan
    df.loc[outlines, column] = np.nan
    return df


####################################################################################################


# Вторая функция: Ограничение и винзоризация для выбросов
def remove_outliers_and_handle_skewness(df, columns, threshold=1.5, cap_percentiles=(0.01, 0.99)):
    """
    Удаляет выбросы на основе асимметрии и применяет ограничение или преобразование.
    """
    df_cleaned = df.copy()

    for col in columns:
        # Обрабатываем асимметрию, применяя логарифмическое преобразование (если сильно асимметрично)
        if df[col].skew() > 1:
            df_cleaned[col] = np.log1p(df_cleaned[col])

        # Рассчитываем IQR для обнаружения выбросов
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1

        # Рассчитываем нижний и верхний пределы для выбросов
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        # Применяем ограничение для выбросов по IQR и заменяем выбросы на медиану или ограниченные значения
        outliers = (df_cleaned[col] < lower_bound) | (df_cleaned[col] > upper_bound)


        # Заменяем выбросы на NaN используя булевую маску
        df_cleaned.loc[outliers, col] = np.nan

        # В качестве альтернативы, применяем винзоризацию, ограничивая значения на указанных процентилях (если нужно более мягкое ограничение)
        lower_cap = df_cleaned[col].quantile(cap_percentiles[0])
        upper_cap = df_cleaned[col].quantile(cap_percentiles[1])
        df_cleaned[col] = df_cleaned[col].clip(lower=lower_cap, upper=upper_cap)

    return df_cleaned


####################################################################################################


# Гибридная функция, которая решает, применять ли ограничение или замену медианой в зависимости от асимметрии и степени выбросов
def hybrid_outlier_handling(df: pd.DataFrame, columns: list, threshold=1.5, multp=3, cap_percentiles=(0.01, 0.99)):
    """
    Гибридная функция для выбора между ограничением или заменой выбросов медианой,
    в зависимости от асимметрии столбца и степени выбросов.
    """
    df_cleaned = df.copy()

    for col in columns:
        # Обрабатываем асимметрию до обработки выбросов
        if df[col].skew() > 1:  # Если сильно асимметрично, сначала применим преобразование
            df_cleaned[col] = np.log1p(df_cleaned[col])

        # Рассчитываем IQR для обнаружения выбросов
        Q1 = df_cleaned[col].quantile(0.25)
        Q3 = df_cleaned[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR

        # Проверяем, есть ли экстремальные выбросы (за пределами 3x IQR)
        if any(df_cleaned[col] < (Q1 - 3 * IQR)) or any(df_cleaned[col] > (Q3 + 3 * IQR)):
            # Если есть экстремальные выбросы, заменяем их на медиану с помощью iqr_filter
            df_cleaned = radical_iqr_filter(df_cleaned, col, lower_bound=True, upper_bound=True, drop_nan_values=True,multp=multp)
        else:
            # В противном случае ограничиваем выбросы с помощью remove_outliers_and_handle_skewness
            df_cleaned = remove_outliers_and_handle_skewness(df_cleaned, [col], threshold=threshold, cap_percentiles=cap_percentiles)

    return df_cleaned


####################################################################################################


def super_train_test_split(df: pd.DataFrame, y: pd.Series):
    '''
    Делит данные на две выборки: 1. строки, значения необходимого нам столбца не имеют пропусков.
                                2. строки, значения необходимого нам столбца имеют пропуски.
    Каждый из этих пунктов так же делиться на две выборки: а) необходимый столбец.
                                                            б) остальные факторы.

    Аргументы:
        df: Pandas DataFrame, состоящий из факторов, инмеющих зависимость с признаком,
            в котором необходимо заполнить пропуски.

        y: Pandas Series, признак, пропуски которого необходимо заполнить.

    небольшой комментарий:
    У нас есть проблема - для заполенния пропусков с помощью какой-либо модели, необходимо,
    чтобы ВСЕ значения в других признаках были заполнены(не было пропусков).
    В противном случае модель ругается, что есть NaNы. Данный цикл устраняет данную проблему,
    временно заполняя пропуски в столбцах на медиану всех значений признака
    (кроме столбца, задача для которого изначально была заполнить пропуски с помощью модели).
    Дальше смотрите по комментариям
    '''
    X = df.copy()

    y_train = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X_train = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков

    idxs = y_train.index # берём иднексы c изначально заполенными значениями
    X_test = X.drop(idxs) # отбрасываем строки с заполненными значениями в нужном нам столбце.
                        # Получается выборка с данными, на основе которых будут
                        # предсказываться пропущенные значения



    return X_train, X_test, y_train, y_temp


####################################################################################################


def split_for_grade(df: pd.DataFrame, target_column: pd.Series): # просто раздел данных на
                                                                 # выборки для обучения и тестирования
    X = df.copy()

    if target_column.name in X.columns:
        X.pop(target_column.name)
    y = target_column
    y1 = y[y.isnull() == False] # отбираем для тренировки те строки, в которых присутсвуют данные
    y_temp = y[y.isnull()] # просто мусор. Полезный

    idxs = y_temp.index # берём иднексы мусора(индексы,
                      # в строках которых есть пропуски, которые необходимо заполнить)
    X = X.drop(idxs) # делаем обучающую выборку из строк, в которых нет пропусков
    X = X.reset_index(drop=True)

    kernel = mf.ImputationKernel(
        data=X,
        random_state=42
    )
    y1 = y1.reset_index(drop=True)

    kernel.mice(iterations=1) # Количество итерация

    # Получаем датафрейм без пропусков
    X = kernel.complete_data()


    X_train, X_test, y_train, y_test = train_test_split(
        X, y1, test_size=len(y_temp) / len(y1), random_state=42)
    return X_train.values, X_test.values, y_train.values, y_test.values

### **Классы**

In [4]:
class SBS():
    """
    Класс для последовательного обратного отбора признаков (Sequential Backward Selection).

    Алгоритм отбирает подмножество наиболее важных признаков,
    оптимизируя модель по метрикам качества (R-квадрат и MSE).

    Аргументы:
        estimator: Модель машинного обучения, которую нужно оптимизировать.
                   Должна поддерживать методы fit и predict.
        k_features: Целевое количество признаков для отбора.
        test_size: Доля данных для тестирования (кросс-валидация).
        random_state: Случайное зерно для воспроизводимости результатов.

    """
    def __init__(self, estimator, k_features,
                test_size=0.25, random_state=42):
        # Создаём копию модели, чтобы случайно не изменить исходную
        self.estimator = clone(estimator)
        self.k_features = k_features
        self.test_size = test_size
        self.random_state = random_state

    def fit(self, X, y, own_split=False):
        """
        Обучает модель SBS и отбирает лучшие признаки.

        Аргументы:
            X: Матрица признаков.
            y: Вектор целевой переменной.
            own_split: Если True, использует пользовательскую функцию split_for_grade для разделения данных.
        """

        # Разделение данных на обучающую и тестовую выборки
        X_train, X_test, y_train, y_test = split_for_grade(X, y)


        dim = X_train.shape[1]

        self.indices_ = list(range(dim))  # Индексы всех признаков
        self.subsets_ = [self.indices_] # Список всех подмножеств признаков

        # Вычисляем R-SQUARED и MSE
        score = self._calc_score(X_train, y_train,
                                    X_test, y_test, self.indices_)
        self.scores_ = [score]

        while dim > self.k_features:
            scores = []
            subsets = []

            # Перебор всех возможных подмножеств с одним удаленным признаком
            for p in combinations(self.indices_, r=dim - 1):
                score = self._calc_score(X_train, y_train,
                                            X_test, y_test, p)
                scores.append(score)
                subsets.append(p)

            # находим подмножества с лучшими значениями метрик
            best = np.argmax([i[0] for i in scores]) #  Выбираем подмножество с наибольшим r2_score,
                                                     # т.к. данная метрика в приоритете. Так же отбор
                                                     # лучшей комбинации будет происходит вне класса
            self.indices_ = subsets[best]
            self.subsets_.append(self.indices_)
            dim -= 1

            self.scores_.append(scores[best])
        self.k_score_ = self.scores_[-1]

        return self

    def transform(self, X):
        """
        Возвращает матрицу признаков с отобранными признаками.

        Аргументы:
            X: Матрица признаков.
        """
        return X[:, self.indices_]

    def _calc_score(self, X_train, y_train, X_test, y_test, indices):
        """
        Вычисляет метрики R-SQUARED и MSE для заданного подмножества признаков.

        Аргументы:
            X_train, y_train, X_test, y_test, indices: Данные для обучения и оценки модели.
        """
        self.estimator.fit(X_train[:, indices], y_train)
        y_pred = self.estimator.predict(X_test[:, indices])

        score = [r2_score(y_test, y_pred), mean_squared_error(y_test, y_pred)]
        return score


## Инициализация моделей и главного дата фрейма

инициализация моделей и методов

In [5]:
knn_model = KNeighborsRegressor(n_neighbors=3)
random_forest = RandomForestRegressor(random_state=42)
scaler = MinMaxScaler()

In [6]:
df_environmental_data = pd.read_csv("analysing_environmental_issues.csv", sep=',') # главный датафрейм

In [7]:
df = df_environmental_data.copy()

## предобработка перед оценкой моделей и предсказыванием значений

Перевод типов признаков в другие, нормализация, удаление дубликатов и обработка выбросов.

In [8]:
df.describe()

,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,stage_2_output_top_vacuum,...,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_danger_gas,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
count,4159.000000,4177.000000,4180.000000,4209.000000,4223.000000,4169.000000,4218.000000,4226.000000,4196.000000,4205.000000,...,4226.000000,4170.000000,4231.000000,4174.000000,4156.000000,4159.000000,934.000000,4240.000000,4240.000000,4240.000000
mean,69.456160,86.674616,404.030844,79.215959,98.476441,56.764406,450.264177,261.478121,94.630858,59.045707,...,109.998318,42.777156,153.448811,20.162808,5.402151,313.779618,0.140139,22.438208,46.346776,1.483019
std,4.032077,31.155280,62.018933,3.027407,8.890578,7.858853,72.004423,43.201651,4.541636,11.180912,...,2.783694,4.472304,1.759867,3.080904,1.074238,104.519417,0.038566,1.243364,13.022949,0.499770
min,50.330000,19.950000,248.760000,66.130000,79.590000,34.070000,260.220000,134.920000,81.050000,33.150000,...,102.330000,25.940000,110.040000,-0.170000,2.350000,65.260000,0.020000,17.280000,0.710000,1.000000
25%,67.030000,64.820000,353.252500,77.430000,91.450000,52.250000,407.800000,230.027500,93.550000,49.910000,...,108.320000,40.960000,152.330000,18.100000,4.740000,245.160000,0.110000,21.680000,40.065000,1.000000
50%,70.030000,82.900000,389.395000,78.820000,97.200000,56.070000,436.960000,259.660000,95.530000,56.470000,...,109.265000,44.100000,153.210000,20.510000,5.500000,303.390000,0.140000,22.580000,47.870000,1.000000
75%,72.330000,105.570000,458.350000,80.850000,103.155000,60.030000,475.602500,290.480000,97.620000,68.780000,...,111.067500,45.860000,153.835000,22.130000,6.140000,366.005000,0.167500,23.280000,55.310000,2.000000
max,79.830000,233.370000,897.290000,105.460000,130.930000,125.360000,1000.750000,579.640000,109.900000,112.380000,...,123.500000,53.650000,157.680000,31.460000,7.980000,725.740000,0.340000,25.480000,107.050000,2.000000


Анализуря данные, выведенные в таблице выше, можно заметить болшие диапазоны значений, которые могут плохо повлиять на обучение модели и предсказание пропущенных значений, поэтому необходимо выполнить нормализацию данных.

известно, что столбец "DateTime" имеет неправильный тип данных, но мы так же его должны нормализовать.

признак "stage_4_output_danger_gas" имеет большое количество пропущенных значений, поэтому будет хорошей идеей заполнить его самым последним в очереде.

In [9]:
df['DateTime'] = pd.to_datetime(df['DateTime'], errors='coerce')
time_diffs = df['DateTime'].diff().dt.total_seconds()
time_diffs = time_diffs.fillna(0)

# нормализуем даты из столбца DateTime
normalized_diffs = scaler.fit_transform(time_diffs.values.reshape(-1, 1)).flatten()

# вычисляет кумулятивную сумму элементов
normalized_times = np.cumsum(normalized_diffs)

# подставляем нормализованные значение
df['DateTime'] = normalized_times

df.pop("stage_4_output_danger_gas") # значений мало, признак на данном этапе бесполезный.

df['work_shift'] = np.where(df['work_shift'] == 1.0, 0, 1)

df = df.drop_duplicates(subset=df.columns[1:], keep=False) # удаляем все дупликаты.

df = pd.DataFrame(scaler.fit_transform(df.copy()), columns=df.columns)
# Очень важно, что дубликаты будут найдены, если игнорировать столбец "DateTime".
df.head(5)

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_input_steam,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift
0,0.000000,0.59322,0.342236,0.347586,0.272311,0.349240,0.173842,0.136767,0.263357,0.569497,...,0.357703,0.298536,0.709130,0.978799,0.608599,0.634103,0.440271,0.512195,0.435584,1.0
1,0.000053,0.59322,0.336754,0.346815,0.254767,0.355084,0.158615,0.127652,0.247234,0.575737,...,0.362257,0.301370,0.719957,0.980688,0.610813,0.637655,0.442754,0.512195,0.416964,1.0
2,0.000107,0.59322,0.331272,0.346029,0.237223,0.360927,0.143280,0.118537,0.231112,0.582322,...,0.366818,0.303732,0.730783,0.982788,0.613342,0.642984,0.445237,0.512195,0.398251,1.0
3,0.000160,0.59661,0.329351,0.346676,0.232138,0.354499,0.167160,0.130339,0.258477,0.570191,...,0.393501,0.297591,0.719235,0.973342,0.617452,0.642984,0.415955,0.512195,0.425992,1.0
4,0.000213,0.60000,0.327476,0.347309,0.227053,0.348072,0.191040,0.142155,0.285843,0.558059,...,0.420192,0.291923,0.707326,0.963896,0.621562,0.642984,0.386658,0.512195,0.453639,1.0


In [224]:
df.isnull().any()

,0
DateTime,False
stage_1_output_konv_avd,True
stage_2_input_water_sum,True
stage_2_output_bottom_pressure,True
stage_2_output_bottom_temp,True
stage_2_output_bottom_temp_hum_steam,True
stage_2_output_bottom_vacuum,True
stage_2_output_top_pressure,True
stage_2_output_top_pressure_at_end,True
stage_2_output_top_temp,True


Всего лишь четыре признака не имеют пропусков в данных. Данное число уменьшится после обработки выбросов до двух.

In [10]:
#Замена выбросов на np.nan
df = hybrid_outlier_handling(df, df.columns[1:-1])

In [11]:
df.isnull().any()

DateTime                                False
stage_1_output_konv_avd                  True
stage_2_input_water_sum                  True
stage_2_output_bottom_pressure           True
stage_2_output_bottom_temp               True
stage_2_output_bottom_temp_hum_steam     True
stage_2_output_bottom_vacuum             True
stage_2_output_top_pressure              True
stage_2_output_top_pressure_at_end       True
stage_2_output_top_temp                  True
stage_2_output_top_vacuum                True
stage_3_input_pressure                   True
stage_3_input_soft_water                 True
stage_3_input_steam                      True
stage_3_output_temp_hum_steam            True
stage_3_output_temp_top                  True
stage_4_input_overheated_steam           True
stage_4_input_polymer                    True
stage_4_input_steam                      True
stage_4_input_water                      True
stage_4_output_dry_residue_avg           True
stage_4_output_product            

## **SBS  для выбора факторов , на основе которых буддет обучаться модель и предсказываться значения**

Данный алгоритм выбирает лучшую комбинацию факторов для каждого столбца, которые имеют пропуски в данных, исходя из оценок KNN алгоритма с помощью метрик R2 и MSE. Полученная комбинация записывается в словарь  **columns_dict_NaN_for_predict** с названием признака в качестве ключа

In [12]:
columns_dict_NaN_for_predict = {} # 'column_name': [best_columns_combination]

In [13]:
# запись изначальных индексов строк в лист и последующий сброс индексов в датфрейме.
# Это необходимое действие, так как mouseforest выдаёт ошибку,
# если нарушен порядок индексов, что и было сделано при удалении дубликатов.
index_list = df.index.tolist()
index_series = pd.Series(index_list)
df = df.reset_index(drop=True)

In [14]:
#инициализируем алгоритм sbs.
sbs = SBS(knn_model, k_features=1)

#поиск лучшей комбинации признаков для каждого столбца в датафрейме.
for column in df.columns:
    if df[column].isnull().any() == False:
        continue

    # подготовка данных
    y = df[column]
    X = df.copy()
    X.pop(column)

    #для корректной работы необходимо перевести названия столбцов в численный вид.
    new_names = [i for i in range(len(df.columns))]

    #скармливаем данные алгоритму sbs.
    sbs.fit(X, y)

    #переименовываем название факторов в численный вид
    X = X.rename(columns=dict(zip(X, new_names)))

    #инициализируем переменные для отбора лучшей комбинации признаков
    best_r2 = -1
    best_mse = float('inf')
    best_pair = None
    lk = -1



    #в sbs.scores_ записываются оценки метрик за все проверенные комбинации
    #поэтому необходимо отобрать лучшие показатели.
    for i, (r2_sc, mse_sc) in enumerate(sbs.scores_):
        #простой, но допустимы отбор
        if r2_sc > best_r2 and mse_sc < best_mse:
            best_r2 = r2_sc
            best_mse = mse_sc

            best_pair = [best_r2, best_mse]

            #создание списка индексов признаков.
            lk = list(sbs.subsets_[sbs.scores_.index([best_r2, best_mse])])

    #Так как при создании списка индексов признаков не учитывается, что был удалён
    #столбец, относительно которого ведутся вычесления, необходимо отредактировать
    #созданный массив.
    if df.columns.get_loc(y.name) in lk:
        index = lk.index(df.columns.get_loc(y.name))
        lk = np.array(lk)

        if index < len(lk) - 1:
            lk = np.concatenate((lk[:index], lk[index:] + 1))
        else:
            lk = lk[:index]
    else:
        for i in range(len(lk)):
            if lk[i] > df.columns.get_loc(y.name):
                lk[i] += 1

    #вывод результатов вычеслений
    print(column)
    print(f"набор индексов лучших факторов: {lk}")
    print(f"Лучшая пара метрик: R2 = {best_pair[0]:.4f}, MSE = {best_pair[1]:.4f}")
    print('=-----------------------------------------------')

    columns_dict_NaN_for_predict[column] = [col for col in list(df.columns[0:][lk]) if col != column]

#вывод итогового словаря
print(columns_dict_NaN_for_predict)


stage_1_output_konv_avd
набор индексов лучших факторов: [3, 5, 9, 11, 12, 14, 17, 18, 19, 22]
Лучшая пара метрик: R2 = 0.9093, MSE = 0.0016
=-----------------------------------------------
stage_2_input_water_sum
набор индексов лучших факторов: [5, 6, 8, 9, 11, 13, 15, 17, 18, 19, 20, 21]
Лучшая пара метрик: R2 = 0.9758, MSE = 0.0002
=-----------------------------------------------
stage_2_output_bottom_pressure
набор индексов лучших факторов: [ 0  2  4  5  6  7  8  9 10 11 17 19 20 21 22]
Лучшая пара метрик: R2 = 0.9967, MSE = 0.0000
=-----------------------------------------------
stage_2_output_bottom_temp
набор индексов лучших факторов: [3, 6, 7, 21]
Лучшая пара метрик: R2 = 0.9937, MSE = 0.0000
=-----------------------------------------------
stage_2_output_bottom_temp_hum_steam
набор индексов лучших факторов: [2, 7, 8, 9, 10, 12, 13, 14, 15, 16, 17]
Лучшая пара метрик: R2 = 0.9831, MSE = 0.0003
=-----------------------------------------------
stage_2_output_bottom_vacuum
набор ин

# Замена пропущенных значений

Для макисимально точного предсказания значений и качественного обучения модели было принято использовать две модели: KNN и RFR. Пропуски в данных, которые будут служить факторами для обучения и предсказания, заменяются на значения с MICE. Будет вестись запись метрик каждой модели для последующего нахождения лучшего варианта замены пропусков в каждом столбце.

Были использованы именно эти модели, потому что KNN показал свою эффективность в отборе факторов для обучения модели, а RFR является отличным вариантом для нахождения нелинейных зависимостей, обработки высокоразмерных данных.

In [15]:
#df

In [16]:
df1 = df.copy() # чек-поинт

In [17]:
df = df1.copy() # загрузка последнего сохранения

Создаём временный датафрейм **df_temp** и заполняем пропуски с помощью MICE. Данный датафрейм будет корректироваться после получения новых предсказаний модели.

In [18]:
df_temp = df.copy()
kernel = mf.ImputationKernel(
    data=df_temp,
    random_state=42
)

kernel.mice(iterations=5)

# Получаем датафрейм без пропусков
df_temp = kernel.complete_data()

In [19]:
#создаём временную копию главного датафрейма.
df_without_nan_by_cnn = df.copy()

#словарь со значениями метрики r2 каждого столбца,
#в котором обнаружены пропуски
metrics_by_cnn = {}

In [20]:
for column in columns_dict_NaN_for_predict.keys(): # для каждого столбца в котором остались пропущенные значения

    X = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy() # все зависимые признаки с столбцом column ()

    y = df_without_nan_by_cnn[column] # наш столбец.

    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)

    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    df_without_nan_by_cnn.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений
    df_temp.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений для более точных предсказаний

    X_for_grade = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy()
    y_for_grade = df_temp[column] # наш столбец.

    X_train, X_test, y_train, y_temp = train_test_split(X_for_grade, y_for_grade, test_size=0.2, random_state=42) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    knn_model.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)
    y_pred = knn_model.predict(X_test) # предсказываем пропущенные значения

    #оценка модели
    r2 = r2_score(y_temp, y_pred)
    mse = mean_squared_error(y_temp, y_pred)

    metrics_by_cnn[column] = [r2] #запись оценку метрики r2 в словарь

    print("-" * 50)
    print(f'{column} -> r2: {r2}, mse: {mse}')

--------------------------------------------------
stage_1_output_konv_avd -> r2: 0.8031894257458316, mse: 0.0035331997272739367
--------------------------------------------------
stage_2_input_water_sum -> r2: 0.9294234275474897, mse: 0.0006035520731615288
--------------------------------------------------
stage_2_output_bottom_pressure -> r2: 0.9404176960653089, mse: 0.0005395853370318336
--------------------------------------------------
stage_2_output_bottom_temp -> r2: 0.9486786615735028, mse: 0.0001472043437171784
--------------------------------------------------
stage_2_output_bottom_temp_hum_steam -> r2: 0.9288415870673188, mse: 0.0019048505460836328
--------------------------------------------------
stage_2_output_bottom_vacuum -> r2: 0.9339951633720945, mse: 0.0004628960081428931
--------------------------------------------------
stage_2_output_top_pressure -> r2: 0.9446661542137663, mse: 0.00023192664316491293
--------------------------------------------------
stage_2_outpu

In [21]:
df = df1.copy() # загрузка последнего сохранения

In [22]:
#аналогичные действия были описаны выше
df_temp = df.copy()
kernel = mf.ImputationKernel(data=df_temp, random_state=42)
kernel.mice(iterations=5)
df_temp = kernel.complete_data()

In [23]:
#создаём временную копию главного датафрейма.
df_without_nan_by_rfr = df.copy()

#словарь со значениями метрики r2 каждого столбца,
#в котором обнаружены пропуски
metrics_by_rfr = {}

In [24]:
for column in columns_dict_NaN_for_predict.keys(): # для каждого столбца в котором остались пропущенные значения
    y = df_without_nan_by_rfr[column] # наш столбец.

    X = df_temp.loc[:, columns_dict_NaN_for_predict[column]].copy()

    X_train, X_test, y_train, y_temp = super_train_test_split(X, y) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку

    random_forest.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)

    y_pred = random_forest.predict(X_test) # предсказываем пропущенные значения

    df_without_nan_by_rfr.loc[X_test.index, column] = y_pred # вставляем предсказания на места пропущенных значений
    df_temp.loc[X_test.index, column] = y_pred

    X_for_grade = df_without_nan_by_rfr.loc[:, columns_dict_NaN_for_predict[column]].copy()
    y_for_grade = df_without_nan_by_rfr[column] # наш столбец.

    X_train, X_test, y_train, y_temp = train_test_split(X_for_grade, y_for_grade, test_size=0.2, random_state=42) # Делим наши данные на
                                                                    # на обучающую и тестовую выборку
    random_forest.fit(X_train, y_train) # обучаем модель knn(для каждого столбца)
    y_pred = random_forest.predict(X_test) # предсказываем пропущенные значения

    #оценка модели
    r2 = r2_score(y_temp, y_pred)
    mse = mean_squared_error(y_temp, y_pred)

    metrics_by_rfr[column] = [r2] #запись оценку метрики r2 в словарь

    print("-" * 50)
    print(f'{column} -> r2: {r2}, mse: {mse}')



--------------------------------------------------
stage_1_output_konv_avd -> r2: 0.7643141128539633, mse: 0.004214853943038354
--------------------------------------------------
stage_2_input_water_sum -> r2: 0.8907918565339559, mse: 0.0009398917630084822
--------------------------------------------------
stage_2_output_bottom_pressure -> r2: 0.8353085245703972, mse: 0.0014913710738292786
--------------------------------------------------
stage_2_output_bottom_temp -> r2: 0.9590051533131233, mse: 0.00011731723662389184
--------------------------------------------------
stage_2_output_bottom_temp_hum_steam -> r2: 0.8722552163549242, mse: 0.0032790466845877613
--------------------------------------------------
stage_2_output_bottom_vacuum -> r2: 0.9368529556558456, mse: 0.00044202421675367367
--------------------------------------------------
stage_2_output_top_pressure -> r2: 0.919942625862151, mse: 0.0003345199309301725
--------------------------------------------------
stage_2_output

In [25]:
# заменяем столбец с пропущенными данными лучшим, опираясь на метрику r2
for column in df.columns[1:-1]: # первые два столцба не имеют пропусков
    if metrics_by_rfr[column] > metrics_by_cnn[column]:
        df[column] = df_without_nan_by_rfr[column]
    else:
        df[column] = df_without_nan_by_cnn[column]



# Сохранение обработанного датафрейма

In [28]:
df = df.set_index(index_series) # заменяем сброшенные индесы на старые для корректной работы.

In [29]:
# добавляем столб stage_4_output_danger_gas
df["stage_4_output_danger_gas"] = df_environmental_data['stage_4_output_danger_gas'].loc[df.index]
#df["stage_4_output_danger_gas"]

In [30]:
df.head(10)

,DateTime,stage_1_output_konv_avd,stage_2_input_water_sum,stage_2_output_bottom_pressure,stage_2_output_bottom_temp,stage_2_output_bottom_temp_hum_steam,stage_2_output_bottom_vacuum,stage_2_output_top_pressure,stage_2_output_top_pressure_at_end,stage_2_output_top_temp,...,stage_3_output_temp_hum_steam,stage_3_output_temp_top,stage_4_input_overheated_steam,stage_4_input_polymer,stage_4_input_steam,stage_4_input_water,stage_4_output_dry_residue_avg,stage_4_output_product,work_shift,stage_4_output_danger_gas
0,0.000000,0.593220,0.294337,0.347586,0.240835,0.349240,0.173842,0.128188,0.263357,0.569497,...,0.261237,0.709130,0.978799,0.608599,0.634103,0.440271,0.512195,0.435584,1.0,NaN
1,0.000053,0.593220,0.290244,0.346815,0.226950,0.355084,0.158615,0.120137,0.247234,0.575737,...,0.263417,0.719957,0.980688,0.610813,0.637655,0.442754,0.512195,0.416964,1.0,NaN
2,0.000107,0.593220,0.286135,0.346029,0.212870,0.360927,0.143280,0.112021,0.231112,0.582322,...,0.265231,0.730783,0.982788,0.613342,0.642984,0.445237,0.512195,0.398251,1.0,NaN
3,0.000160,0.596610,0.284691,0.346676,0.208751,0.354499,0.167160,0.122518,0.258477,0.570191,...,0.260509,0.719235,0.973342,0.617452,0.642984,0.415955,0.512195,0.425992,1.0,0.16
4,0.000213,0.600000,0.283280,0.347309,0.204615,0.348072,0.191040,0.132917,0.285843,0.558059,...,0.256131,0.707326,0.963896,0.621562,0.642984,0.386658,0.512195,0.453639,1.0,NaN
5,0.000266,0.600000,0.311195,0.347956,0.217176,0.359758,0.188411,0.135066,0.284021,0.578856,...,0.249528,0.705161,0.967884,0.682896,0.621670,0.359526,0.512195,0.476585,1.0,NaN
6,0.000320,0.633898,0.338351,0.348604,0.229581,0.371250,0.185672,0.137199,0.282223,0.599653,...,0.243252,0.702995,0.971662,0.744230,0.598579,0.332395,0.512195,0.499530,1.0,NaN
7,0.000373,0.708475,0.338652,0.348866,0.242033,0.364044,0.188082,0.138670,0.283167,0.592374,...,0.246949,0.714543,0.974391,0.730319,0.600355,0.385613,0.573171,0.487493,1.0,0.15
8,0.000426,0.779661,0.338953,0.349143,0.254136,0.356642,0.190382,0.140138,0.284089,0.585442,...,0.250264,0.725731,0.977120,0.716408,0.600355,0.438817,0.646341,0.475456,1.0,NaN
9,0.000480,0.779661,0.338084,0.348650,0.242632,0.359174,0.171979,0.130241,0.262682,0.586482,...,0.256862,0.725009,0.976280,0.639583,0.611012,0.437621,0.646341,0.449878,1.0,NaN


In [1]:
df.to_csv("../analysing_environmental_with_normalized_values.csv", encoding='utf-8', index=False)

NameError: name 'df' is not defined